# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import gmaps

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load the csv exported in Part I to a DataFrame
weather_df = pd.read_csv("cities.csv")
weather_df.head()



,city,lat,lng,max_temp,humidity,cloudiness,windspeed
0,Siniscola,40.57,9.69,48.20,87,0,5.82
1,Makakilo City,21.35,-158.09,78.80,100,90,3.36
2,Upernavik,72.79,-56.15,-35.79,91,57,9.44
3,Rikitea,-23.12,-134.97,75.90,72,42,16.87
4,Juneau,58.30,-134.42,28.40,86,90,5.39


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Store latitude and longitude in locations
locations = weathers_df[["Lat","Lng"]]

# Filla NaN values
#locations = locations.dropna()
locations.head()


NameError: name 'weathers_df' is not defined

In [6]:
# Defining Humidity as weight
weights = weather_df["humidity"].astype(float)

In [7]:
#Set up your map
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

NameError: name 'locations' is not defined

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow new DF to the 3 criterias defined for the exercise
weather2 = weather_df
weather2 = weather2[(weather2["max_temp"] > 70)]
weather2 = weather2[(weather2["humidity"] < 80)] 
weather2 = weather2[(weather2["windspeed"] < 10)]
weather2 = weather2[(weather2["cloudiness"] == 0)]

# drop null
weather2 = weather2.dropna()
weather2.count()

city          12
lat           12
lng           12
max_temp      12
humidity      12
cloudiness    12
windspeed     12
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Store into variable named hotel_df.
hotel_df = weather2

# set up additional column to hold information
hotel_df['Hotel Name'] = " "
hotel_df.head()

,city,lat,lng,max_temp,humidity,cloudiness,windspeed,Hotel Name
60,Atuona,-9.80,-139.03,81.03,70,0,5.41,
81,Cockburn Town,21.46,-71.14,77.20,68,0,7.29,
219,Edd,13.93,41.69,82.00,51,0,8.97,
222,Port Shepstone,-30.74,30.45,82.26,56,0,3.69,
367,Filadelfia,-22.35,-60.04,73.24,36,0,4.59,


In [10]:
# find the closest hotel of each type to coordinates
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key,}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)

    # convert to json
    data = response.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

NameError: name 'g_key' is not defined

In [11]:
hotel_df


,city,lat,lng,max_temp,humidity,cloudiness,windspeed,Hotel Name
60,Atuona,-9.80,-139.03,81.03,70,0,5.41,
81,Cockburn Town,21.46,-71.14,77.20,68,0,7.29,
219,Edd,13.93,41.69,82.00,51,0,8.97,
222,Port Shepstone,-30.74,30.45,82.26,56,0,3.69,
367,Filadelfia,-22.35,-60.04,73.24,36,0,4.59,
378,Chum Phae,16.54,102.10,85.51,39,0,4.43,
442,Ankazoabo,-22.29,44.51,84.45,51,0,7.40,
452,San Quintin,17.54,120.52,84.16,51,0,6.49,
472,Jaisalmer,26.91,70.91,71.17,24,0,8.08,
501,San Patricio,-26.98,-56.83,70.97,68,0,4.65,


In [12]:
# Add marker layer ontop of heat map
fig = gmaps.figure(layout={
        'padding': '3px',
        'border': '1px solid black'
})
locations2 = hotel_df[["Lat", "Lng"]]
fig.add_layer(gmaps.heatmap_layer(locations2))
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_df = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Display Map
fig


# Display Map

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"